# Importing packages

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
from gtsimulation.Algos import BunemanBorisSimulator, RungeKutta6Simulator
from gtsimulation.Global import Regions, Constants, Units
from gtsimulation.MagneticFields.Magnetosphere import Dipole
from gtsimulation.Particle import Generators, Flux

# General GT settings

In [ ]:
date = datetime(2008, 1, 1)
region = Regions.Magnetosphere
b_field = Dipole(date=date)
medium = None

particle = Flux(
    Spectrum=Generators.Spectrums.Monolines(energy=10 * Units.MeV),
    Distribution=Generators.Distributions.UserInput(
        R0=np.array([5 * Units.RE, 0, 0]),
        V0=[0, 1, 0]
    ),
    Names="proton",
    Nevents=1
)

use_decay = False
nuclear_interaction = None

total_time = 10  # total time [s]
n_steps = 20000
dt = total_time / n_steps
break_conditions = None

save = [1, {"Coordinates": True, "Velocities": True, "Energy": True, "Bfield": True}]
output = None

verbose = True

# Buneman–Boris scheme

In [ ]:
simulator_BB = BunemanBorisSimulator(
    Bfield=b_field,
    Region=region,
    Medium=medium,
    Particles=particle,
    InteractNUC=nuclear_interaction,
    UseDecay=use_decay,
    Date=date,
    Step=dt,
    Num=n_steps,
    BreakCondition=break_conditions,
    Save=save,
    Output=output,
    Verbose=verbose
)

track_BB = simulator_BB()[0][0]
r_BB = track_BB["Track"]["Coordinates"]
v_BB = track_BB["Track"]["Velocities"]
T_BB = track_BB["Track"]["Energy"]
B_BB = track_BB["Track"]["Bfield"]

# Runge–Kutta 6th scheme

In [ ]:
simulator_RK6 = RungeKutta6Simulator(
    Bfield=b_field,
    Region=region,
    Medium=medium,
    Particles=particle,
    InteractNUC=nuclear_interaction,
    UseDecay=use_decay,
    Date=date,
    Step=dt,
    Num=n_steps,
    BreakCondition=break_conditions,
    Save=save,
    Output=output,
    Verbose=verbose
)

track_RK6 = simulator_RK6()[0][0]
r_RK6 = track_RK6["Track"]["Coordinates"]
v_RK6 = track_RK6["Track"]["Velocities"]
T_RK6 = track_RK6["Track"]["Energy"]
B_RK6 = track_RK6["Track"]["Bfield"]

# Comparison of trajectories

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.subplots()

ax.scatter(5, 0, label="Initial position", color="black")
ax.plot(*r_BB.T[:2] / Units.RE, label="BB")
ax.plot(*r_RK6.T[:2] / Units.RE, label="RK6")

ax.set_xlim(0, 7)
ax.set_ylim(-6, 1)
ax.set_xlabel("X [RE]")
ax.set_ylabel("Y [RE]")
ax.set_aspect("equal")
ax.grid(True, linestyle="--", alpha=0.8)
ax.legend()

plt.show()

# Comparison of Larmor radii

In [ ]:
from gtsimulation.MagneticFields.Magnetosphere.Additions import GetLarmorRadius

In [ ]:
M = track_BB["Particle"]["M"]
Z = track_BB["Particle"]["Ze"]

B_unit_BB = B_BB / np.linalg.norm(B_BB, axis=1)[:, None]
v_dot_B_BB = np.sum(v_BB * B_unit_BB, axis=1)
pitch_BB = np.rad2deg(np.arccos(v_dot_B_BB))  # clearly get a pitch angle of 90 degrees
LR_BB = GetLarmorRadius(T_BB, np.linalg.norm(B_BB, axis=1), Z, M * Units.MeV2kg, pitch_BB)

B_unit_RK6 = B_RK6 / np.linalg.norm(B_RK6, axis=1)[:, None]
v_dot_B_RK6 = np.sum(v_RK6 * B_unit_RK6, axis=1)
pitch_RK6 = np.rad2deg(np.arccos(v_dot_B_RK6))  # clearly get a pitch angle of 90 degrees
LR_RK6 = GetLarmorRadius(T_RK6, np.linalg.norm(B_RK6, axis=1), Z, M * Units.MeV2kg, pitch_RK6)

In [ ]:
fig = plt.figure(figsize=(12, 4))
ax = fig.subplots()

ax.plot(LR_BB / Units.RE, label="BB")
ax.plot(LR_RK6 / Units.RE, label="RK6")

ax.axhline(y=np.min(LR_BB / Units.RE), linestyle=":")
ax.axhline(y=np.max(LR_BB / Units.RE), linestyle=":")

ax.set_xlabel("Number of steps")
ax.set_ylabel("R$_L$ [RE]")
ax.legend(loc="center left")

plt.show()